In [31]:
import sys
from hand import Hand
from utils import get_possible_slots
from eval7 import Deck, Card, handtype, evaluate
import random
from random import sample
from itertools import product, permutations, combinations
from tqdm import tqdm

import numpy as np

In [32]:
def place_cards(hand, cards_to_place, slots_):
    free_slots = set(permutations(slots_, 2))
    cards_stack = combinations(cards_to_place, 2)

    hands_arr = []

    for cards, slots in product(cards_stack, free_slots):
        temp_hand = hand.copy()
        temp_hand.add_card(cards[0], slots[0])
        temp_hand.add_card(cards[1], slots[1])
        temp_hand.last_update = (cards, slots)
        dead_card = set(cards_to_place) - set(cards)
        hands_arr.append((temp_hand))

    return hands_arr

def is_drawing_dead(hand):
    eval_front = evaluate(hand.front)
    eval_mid = evaluate(hand.mid)
    eval_back = evaluate(hand.back)

    if len(hand.back) == 5 and (eval_mid > eval_back or eval_front > eval_back):
        return 1

    if len(hand.mid) == 5 and eval_front > eval_mid:
        return 1

def solve_2_cards(hand, cards, result='score'):
    max_score = -6
    if result == 'arr':
        res_arr = []

    slots = get_possible_slots(hand)
    for current_hand in place_cards(hand, cards, slots):
        current_hand.evaluate_hand()
        score = current_hand.fantasy_score()
        if current_hand.is_fantasy:
            score += 8

        if score >= max_score:
            max_score = score
            
            if result == 'arr':
                res_arr = [(cards[0], slots[0]), (cards[1], slots[1])]

    if result == 'score':
        return max_score
    elif result == 'arr':
        return res_arr

def solve_4_cards(hand, hole_cards):
    values_dic = {}

    used_cards = hand.front + hand.mid + hand.back + list(hole_cards) + hand.dead_cards
    temp_deck = tuple(card for card in Deck().cards if card not in used_cards)

    slots = get_possible_slots(hand)
    for current_hand in place_cards(hand, hole_cards, slots):
        cards, slots = current_hand.last_update
        values_dic.setdefault((cards[0], slots[0], cards[1], slots[1]), 0)

        for counter, last_street_cards in enumerate(combinations(temp_deck, 3)):
            score = solve_2_cards(current_hand, last_street_cards)
            values_dic[(cards[0], slots[0], cards[1], slots[1])] += score

        values_dic[(cards[0], slots[0], cards[1], slots[1])] /= (counter + 1)

    return max(values_dic, key=values_dic.get)

def simulate_4_cards(hand, cards, n_sim, result='arr'):
        used_cards = hand.front + hand.mid + hand.back + list(cards) + hand.dead_cards
        temp_deck = tuple(card for card in Deck().cards if card not in used_cards)

        values_dic = {}

        slots = get_possible_slots(hand)
        for current_hand in place_cards(hand, cards, slots):
            cards, slots = current_hand.last_update
            
            if is_drawing_dead(current_hand):
                values_dic[(cards[0], slots[0], cards[1], slots[1])] = -6
            else:
                values_dic.setdefault((cards[0], slots[0], cards[1], slots[1]), 0)
                all_combinations = list(combinations(temp_deck, 3))
                combos = sample(all_combinations, n_sim)

                for counter, last_street_cards in enumerate(combos):
                    score = solve_2_cards(current_hand, last_street_cards)
                    values_dic[(cards[0], slots[0], cards[1], slots[1])] += score

                values_dic[(cards[0], slots[0], cards[1], slots[1])] /= (counter + 1)

        if result == 'score':
            return values_dic
        elif result == 'arr':
            return max(values_dic, key=values_dic.get)

def simulate_6_cards(hand, cards, n_sim_4, n_sim_6, result='arr'):
        used_cards = hand.front + hand.mid + hand.back + list(cards) + hand.dead_cards
        temp_deck = tuple(card for card in Deck().cards if card not in used_cards)

        values_dic = {}
        slots = get_possible_slots(hand)
        
        for current_hand in place_cards(hand, cards, slots):
            cards, slots = current_hand.last_update
            
            if is_drawing_dead(current_hand):
                values_dic[(cards[0], slots[0], cards[1], slots[1])] = -6
            else:
                values_dic.setdefault((cards[0], slots[0], cards[1], slots[1]), 0)

                all_combinations = list(combinations(temp_deck, 3))
                combos = sample(all_combinations, n_sim_6)
                
                for counter, street_4_cards in enumerate(combos):
                    res = simulate_4_cards(current_hand, street_4_cards, n_sim_4, result='score')
                    values_dic[(cards[0], slots[0], cards[1], slots[1])] += max(res.values())

                values_dic[(cards[0], slots[0], cards[1], slots[1])] /= (counter + 1)

        if result == 'score':
            return values_dic
        elif result == 'arr':
            return max(values_dic, key=values_dic.get)
    
def simulate_8_cards(hand, cards, n_sim_4, n_sim_6, n_sim_8, result='arr'):
        used_cards = hand.front + hand.mid + hand.back + list(cards) + hand.dead_cards
        temp_deck = tuple(card for card in Deck().cards if card not in used_cards)

        values_dic = {}
        slots = get_possible_slots(hand)

        for current_hand in place_cards(hand, cards, slots):
            cards, slots = current_hand.last_update
            values_dic.setdefault((cards[0], slots[0], cards[1], slots[1]), 0)

            all_combinations = list(combinations(temp_deck, 3))
            combos = sample(all_combinations, n_sim_8)
            
            for counter, street_cards in enumerate(combos):
                counter += 1
                res = simulate_6_cards(current_hand, street_cards, n_sim_4, n_sim_6, result='score')

                values_dic[(cards[0], slots[0], cards[1], slots[1])] += max(res.values())

            values_dic[(cards[0], slots[0], cards[1], slots[1])] /= (counter +1)

        if result == 'score':
            return values_dic
        elif result == 'arr':
            return max(values_dic, key=values_dic.get)

In [33]:
class OFCRandomAgent():
    """Place cards at random"""

    def place_cards(self, cards, hand):
        if len(cards) == 3:
            cards_to_place = random.sample(cards, 2)
            hand.dead_cards.append(list(set(cards) - set(cards_to_place))[0])
        else:
            cards_to_place = cards
            
        res = []
        
        empty_cells = {0: 3 - len(hand.front), 1: 5 - len(hand.mid), 2: 5 - len(hand.back)}
        
        for card in cards_to_place:
            space = [k for k, v in empty_cells.items() if v > 0]
            row = random.choice(space)
            empty_cells[row] -= 1
            
            res.append((card, row))
                                                
        return res
    
    
class Solver():    
    def place_cards(self, cards, hand):
        n_cards = len(hand.front + hand.mid + hand.back)
        
        if n_cards == 9:
            res = solve_4_cards(hand, cards)
            return ((res[0], res[1]), (res[2], res[3]))
        elif n_cards == 11:
            return solve_2_cards(hand, cards, result='arr')
        
        if len(cards) == 3:
            cards_to_place = random.sample(cards, 2)
            hand.dead_cards.append(list(set(cards) - set(cards_to_place))[0])
        else:
            cards_to_place = cards            
            
        res = []
        
        empty_cells = hand.get_empty_cells()
        
        for card in cards_to_place:
            space = [k for k, v in empty_cells.items() if v > 0]
            row = random.choice(space)
            empty_cells[row] -= 1
            
            res.append((card, row))
                                                            
        return res
    
    
class Sim():
    def __init__(self, n_sim_4=None, n_sim_6=None, n_sim_8=None):
        self.n_sim_4 = n_sim_4
        self.n_sim_6 = n_sim_6
        self.n_sim_8 = n_sim_8
        
    def place_cards(self, cards, hand):
        n_cards = len(hand.front + hand.mid + hand.back)
        
        if self.n_sim_8 and n_cards == 5:
            res = simulate_8_cards(hand, cards, self.n_sim_4, self.n_sim_6, self.n_sim_8)
            return ((res[0], res[1]), (res[2], res[3]))
        elif self.n_sim_6 and n_cards == 7:
            res = simulate_6_cards(hand, cards, self.n_sim_4, self.n_sim_6)
            return ((res[0], res[1]), (res[2], res[3]))
        elif self.n_sim_4 and n_cards == 9:
            res = simulate_4_cards(hand, cards, self.n_sim_4)
            return ((res[0], res[1]), (res[2], res[3]))
        elif n_cards == 11:
            return solve_2_cards(hand, cards, result='arr')
        
        if len(cards) == 3:
            cards_to_place = random.sample(cards, 2)
            hand.dead_cards.append(list(set(cards) - set(cards_to_place))[0])
        else:
            cards_to_place = cards            
            
        res = []
        
        empty_cells = hand.get_empty_cells()
        
        for card in cards_to_place:
            space = [k for k, v in empty_cells.items() if v > 0]
            row = random.choice(space)
            empty_cells[row] -= 1
            
            res.append((card, row))
                                                                        
        return res

In [34]:
class OFCHumanAgent():
    def place_cards(self, cards, hand):
        while True:
            place_list = []
            dead_card = None
            empty_cells = {-1: 0, 0: 3 - len(hand.front), 1: 5 - len(hand.mid), 2: 5 - len(hand.back)}

            for card in cards:
                while True:
                    try:
                        row = int(input(f'Выберите номер ряда для карты {card}. Чтобы пропустить карту введите -1: '))
                    except(ValueError):
                        print('Номер ряда должен быть числом.')
                        continue
                    except KeyboardInterrupt:
                        sys.exit(0)

                    if row not in (-1, 0, 1, 2):
                        print('Введите корректный номер ряда:')
                        continue
                    if row == -1:
                        if sum(empty_cells.values()) > 8:
                            print('В первом раунде необходимо выложить все карты.')
                            continue
                        dead_card = card
                        break
                    else:
                        empty_cells[row] -= 1

                    if empty_cells[row] < 0:
                        print('Ряд заполнен! Введите корректный номер ряда:')
                        continue

                    place_list.append((card, row))

                    break

            if len(hand.front + hand.mid + hand.back) >= 5 and len(place_list) != 2:
                print('Необходимо выложить на доску ровно 2 карты.')
                continue

            break
        
        if dead_card:
            hand.dead_cards.append(dead_card)
        
        return place_list

In [35]:
class OnePlayerGame:
    def __init__(self, agent = None, verbose = False):
        self.hand = Hand()
        self.deck = Deck()
        self.deck.shuffle()
        self.agent = agent
        self.verbose = verbose
        
        self.placed_cards = []

    def __print_cards(self, cards):
        print([x.__str__() for x in cards])

    def __print_hand(self, hand):
        len_front = 3 - len(hand.front)
        len_mid = 5 - len(hand.mid)
        len_back = 5 - len(hand.back)

        temp_front = [card.__str__() for card in hand.front] + ['#'] * len_front
        temp_mid = [card.__str__() for card in hand.mid] + ['#'] * len_mid
        temp_back = [card.__str__() for card in hand.back] + ['#'] * len_back

        print(temp_front)
        print(temp_mid)
        print(temp_back)

    def __place_card(self, cards):
        place_list = self.agent.place_cards(cards, self.hand)

        for card, row in place_list:
            self.hand.add_card(card, row)
        
    def play_street(self):
        if self.hand.front + self.hand.mid + self.hand.back:
            cards = self.deck.deal(3)
        else:
            cards = self.deck.deal(5)
        if self.verbose:
            self.__print_hand(self.hand)
            print('*' * 10)
            print('Cards:')
            self.__print_cards(cards)
        self.__place_card(cards)

In [ ]:
def main(agent, verbose=False):
    game = OnePlayerGame(agent=agent, verbose=verbose)
    for _ in range(5):
        game.play_street()

    game.hand.evaluate_hand()
    if verbose:
        print('*' * 10)
        game.hand.print_hand()
            
    return game.hand.fantasy_score()

In [ ]:
agent = Sim(10)
main(agent)

In [ ]:
# res = []
agent = Sim(5, 10, 10)

# main(agent)

for _ in tqdm(range(150)):
    res.append(main(agent))
    
print(np.mean(res))

In [ ]:
len(res)

In [ ]:
# average random score: -4.11 for 1_000_000 samples
# Solver_2 score: -4.12 for 1_000_000 samples

# Solver_4 score: -2.76 for 1_000 samples, 33 min
# Sim_4, 500 sims, score: -2.78 for 1_000 samples, 01:47 min
# Sim_4, 100 sims, score: -2.89 for 1_000 samples, 26 sec

# Sim_6, 50-20 sims, score: -1.81 for 1_000 samples, 1:48 hour
# Sim_6, 10-10 sims, score: -1.71 for 1_000 samples, 25:13 min

# Sim_8, 5-10-10 sims, score: -0.99 for 200 samples, 17:42 hour

In [ ]:
len(list(combinations(range(38), 3)))

In [36]:
h = Hand([Card('Ad'), Card('Qd')], 
         [Card('6d'), Card('2h'), Card('2d'), Card('4s'),], 
         [Card('9d'), Card('7s'), Card('9c')])
cards = (Card('4h'), Card('9h'), Card('2c'))
dead_cards = [Card('Ac'), Card('4c')]

In [104]:
solve_4_cards(h, cards)

(Card("9h"), 2, Card("2c"), 1)

In [30]:
%time solve_4_cards(h, cards)

CPU times: total: 4.05 s
Wall time: 5.78 s


(Card("9h"), 2, Card("2c"), 1)

In [ ]:
# original: 5.89 s
# royalty calculator as functions, not a class: 5.66 s
# cython royalty calculator: 5.1 s

In [ ]:
# Card("9h"), 2, Card("2c"), 1

In [37]:
simulate_4_cards(h, cards, 100)

(Card("9h"), 2, Card("2c"), 1)

In [ ]:
value_dic = {'opt': [], '10': [], '20': [], '50': [], '100': [], '200': [], '500': []}

for _ in tqdm(range(10_000)):
    hand = Hand()
    deck = Deck()
    deck.shuffle()

    base_agent = OFCRandomAgent()
    cards = deck.deal(5)
    place_list = base_agent.place_cards(cards, hand)

    for card, row in place_list:
        hand.add_card(card, row)

    for _ in range(2):
        cards = deck.deal(3)
        place_list = base_agent.place_cards(cards, hand)

        for card, row in place_list:
            hand.add_card(card, row)
    # print('Original hand:')
    # hand.print_hand()
    # print()
    
    cards_4 = deck.deal(3)
    cards_5 = deck.deal(3)
    # print('Cards:', cards_4)
    
    
    for agent in [Solver_4(), Sim_4(10), Sim_4(20), Sim_4(50), Sim_4(100), Sim_4(200), Sim_4(500)]:    
        temp_hand = hand.copy()
        place_list = agent.place_cards(cards_4, temp_hand)
        for card, row in place_list:
            temp_hand.add_card(card, row)

        place_list = solve_2_cards(temp_hand, cards_5, result='arr')
        for card, row in place_list:
            temp_hand.add_card(card, row)


        # temp_hand.print_hand()
        temp_hand.evaluate_hand()
        score = temp_hand.fantasy_score()
        
        if agent.__class__.__name__ == 'Solver_4':
            value_dic['opt'].append(score)
        elif agent.__class__.__name__ == 'Sim_4':
            if agent.__dict__['n_sim'] == 10:
                value_dic['10'].append(score)
            elif agent.__dict__['n_sim'] == 20:
                value_dic['20'].append(score)
            elif agent.__dict__['n_sim'] == 50:
                value_dic['50'].append(score)
            elif agent.__dict__['n_sim'] == 100:
                value_dic['100'].append(score)
            elif agent.__dict__['n_sim'] == 200:
                value_dic['200'].append(score)
            elif agent.__dict__['n_sim'] == 500:
                value_dic['500'].append(score)
        # print('*'*10)
    # print('*'*20)

In [ ]:
print('regret of 10 sims:', np.mean(np.array(value_dic['opt']) - np.array(value_dic['10'])))
print('regret of 20 sims:', np.mean(np.array(value_dic['opt']) - np.array(value_dic['20'])))
print('regret of 50 sims:', np.mean(np.array(value_dic['opt']) - np.array(value_dic['50'])))
print('regret of 100 sims:', np.mean(np.array(value_dic['opt']) - np.array(value_dic['100'])))
print('regret of 200 sims:', np.mean(np.array(value_dic['opt']) - np.array(value_dic['200'])))
print('regret of 500 sims:', np.mean(np.array(value_dic['opt']) - np.array(value_dic['500'])))

In [ ]:
def get_possible_slots(hand):
    front_slots = 3 - len(hand.front)
    mid_slots = 5 - len(hand.mid)
    back_slots = 5 - len(hand.back)

    return (0,) * front_slots + (1,) * mid_slots + (2,) * back_slots


def place_cards(hand, cards_to_place, slots_):
    free_slots = set(permutations(slots_, 2))
    cards_stack = combinations(cards_to_place, 2)
    # print(len(list(cards_stack)))

    hands_arr = []

    for cards, slots in product(cards_stack, free_slots):
        temp_hand = hand.copy()
        temp_hand.add_card(cards[0], slots[0])
        temp_hand.add_card(cards[1], slots[1])
        temp_hand.last_update = (cards, slots)
        hands_arr.append(temp_hand)

    return hands_arr

In [ ]:
%%timeit
h = Hand([Card('Ad'), Card('Qd')], 
         [Card('6d'), Card('2h'), Card('2d'), Card('4s'),], 
         [Card('9d'), Card('7s'), Card('9c')])
cards = (Card('4h'), Card('9h'), Card('2c'))

slots = get_possible_slots(h)
place_cards(h, cards, slots)

In [ ]:
# 49.8 µs ± 1.6 µs

In [ ]:
%load_ext Cython

In [ ]:
%%cython -a
#cython: language_level=3, boundscheck=False, wraparound=False

from itertools import product, combinations, permutations

cpdef object place_cards(object hand, tuple cards_to_place, tuple slots_):
    cdef:
        set free_slots
        tuple cards_stack
        list hands_arr = []
        object temp_hand
    
    free_slots = set(permutations(slots_, 2))
    cards_stack = tuple(combinations(cards_to_place, 2))

    for cards, slots in product(cards_stack, free_slots):
        temp_hand = hand.copy()
        temp_hand.add_card(cards[0], slots[0])
        temp_hand.add_card(cards[1], slots[1])
        temp_hand.last_update = (cards, slots)
        hands_arr.append(temp_hand)

    return hands_arr

In [ ]:
# Original: 498 ns ± 9.24 ns
# Compiled with ctypes: 308 ns ± 7.68 ns  
# 

In [ ]:
from hand import Hand
from eval7 import Deck, Card, handtype, evaluate
h = Hand([Card('Ad')], 
         [Card('6d'), Card('2h'), Card('2d'), Card('4s'),], 
         [Card('9d'), Card('7s')])

In [ ]:
%timeit get_possible_slots(h)

In [ ]:
# %%timeit
h = Hand([Card('Ad'), Card('Qd')], 
         [Card('6d'), Card('2h'), Card('2d'), Card('4s'),], 
         [Card('9d'), Card('7s'), Card('9c')])
cards = (Card('4h'), Card('9h'), Card('2c'))

slots = get_possible_slots(h)
place_cards(h, cards, slots)

In [ ]:
len(place_cards(h, cards, slots))

In [ ]:
%timeit [(a,b) for a in cards_arr for b in free_slots]

In [ ]:
%timeit product(cards_arr, free_slots)

In [ ]:
def place_cards(hand, cards_to_place, slots_):
    free_slots = set(permutations(slots_, 2))

    hands_arr = []
    used_cards = []
    cards_arr = []

    for perm in permutations(cards_to_place, 3):
        dead_card = perm[2]
        if dead_card in used_cards:
            continue

        used_cards.append(dead_card)
        cards_arr.append(perm)

    for cards, slots in product(cards_arr, free_slots):
        temp_hand = hand.copy()
        dead_cards = set((0,1,2)) - set(slots)
        if len(dead_cards) == 1:
            temp_hand.dead_cards.append(cards[dead_cards.pop()])
        elif len(dead_cards) == 2:
            temp_hand.dead_cards.append(cards[dead_cards.pop()])
            temp_hand.dead_cards.append(cards[dead_cards.pop()])
        temp_hand.add_card(cards[0], slots[0])
        temp_hand.add_card(cards[1], slots[1])
        temp_hand.last_update = (cards, slots)
        hands_arr.append(temp_hand)

    return hands_arr

In [ ]:
def place_cards(hand, cards_to_place, slots_):
    free_slots = set(permutations(slots_, 2))
    cards_stack = combinations(cards_to_place, 2)

    hands_arr = []

    for cards, slots in product(cards_stack, free_slots):
        temp_hand = hand.copy()
        temp_hand.add_card(cards[0], slots[0])
        temp_hand.add_card(cards[1], slots[1])
        
        temp_hand.last_update = (cards, slots)
        hands_arr.append(temp_hand)

    return hands_arr

In [ ]:
h = Hand([Card('Ad'), Card('Qd')], 
         [Card('6d'), Card('2h'), Card('2d'), Card('4s'),], 
         [Card('9d'), Card('7s'), Card('9c')])
cards = (Card('4h'), Card('9h'), Card('2c'))

slots = get_possible_slots(h)
z  = place_cards(h, cards, slots)

In [77]:
used_cards = h.front + h.mid + h.back + list(cards) + h.dead_cards


In [86]:
len(set(Deck().cards) - set(used_cards))


40

In [81]:
%timeit set(Deck().cards).difference(set(used_cards))

16.9 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [87]:
%timeit (card for card in Deck().cards if card not in used_cards)

13.2 µs ± 69.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
# 17.1 µs ± 142 n
# 16.9 µs ± 158 ns
# 